# Resumen clase anterior


<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="35%" align="left">
<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_matriz.jpg" width="55%" align="rigth"/>

Calcular el valor de los flujos metabólicos usando numpy.

1. Asumiendo que el flujo limitante es E1=10 [mmol/gDW/h] y que el organismo maximiza la producción de CH4.
2. Igual que 1 pero maximizando la producción de F420.

In [ ]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

# E1,E2,E3,E4,E5,R1,R2,R3

A = np.array([
[1,0,0,0,0,-1,0,0],
[0,1,0,0,0,-1,-1,0],
[0,0,0,0,0,1,-1,1],
[0,0,0,-2,0,-1,1,1],
[0,0,-1,0,0,0,1,-1],
[0,0,0,0,-1,0,0,1]])        

b = np.array([0,0,0,0,0,0])
c = np.array([0,0,1,0,0,0,0,0])

LB=[10,0,0,0,0,0,0,0]       # [0,0,0,0,0,0]
UB=[10,1000000,1000000,1000000,1000000,1000000,1000000,1000000]     # ya sabes la idea

res = linprog(-c, A_eq=A, b_eq=b, bounds=list(zip(LB,UB)))
print('Optimal value:', res.fun, '\nX:', res.x)


In [ ]:
list(zip(LB,UB))

# Clase 4

## Contenidos

* Resolver problemas de ingeniería metabólica a escala genómica
* Introducción a cobrapy



# Introducción a cobra


Para modelar el metabolismo celular a escala genómica utilizaremos la librería ```cobra```. La documentación oficial de cobra se encuentra disponible [aquí](https://cobrapy.readthedocs.io/en/latest/index.html). Hoy veremos los comandos escenciales de cobra y construiremos nuestra propia red metábolica.

## Instalación


Para instalar ```cobra``` debes abrir el terminal de anaconda (si usas Windows, busca Anaconda Prompt ) y escribir en el siguiente comando:

```python
pip install cobra --user
```

Para probar que la librería se instlado correctamente escribe el siguiente comando en la próxima celda:

```python
import cobra
```

Si no hay mensajes de error, entonces ya tienes una versión funcional de cobra.

## Comandos básicos


Comezaremos cargado ```cobra``` y determinando que version está instalada (esto es útil para consultar la documentación adecuada; diferentes versiones de cobra tienen diferentes documentaciones)

In [ ]:
import cobra
cobra.__version__

## Redes metabólicas a escala genómica


En Inglés Genome-scale metabolic networks son redes metabólicas que incluyen todas las reacciones metabólicas presentes en el genoma del organismo bajo estudio.

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase4_eColiMetabolis.png" width="45%" align="left">

En el caso de *Escherichia coli* la red consiste en  reacciones y metabolitos.

["Fuente KEGG"](https://www.genome.jp/kegg-bin/show_pathway?eco01100+M00125)

In [ ]:
import cobra.test

# "ecoli" and "salmonella" are also valid arguments
model = cobra.test.create_test_model("ecoli")

Veamos cuantas reacciones y metabolitos tiene "ecoli"

In [ ]:
print( "Número de reacciones: " , len(model.reactions) )
print( "Número de metabolitos: ", len(model.metabolites) )

Podemos inspeccionar reacciones individualmente, por ejemplo la reacción 30:

In [ ]:
print( model.reactions[30].reaction )

Ver las primeras 10 reacciones:

In [ ]:
for i in range(10):
    print( model.reactions[i] )

## Nota

¿Cuál es tu interpretación de la reacción número 7?

También podemos identificar una reacción por su nombre:

In [ ]:
pgi = model.reactions.get_by_id("PGI")
print( pgi )
print( pgi.reaction )
print( pgi.name )


print(pgi.lower_bound, "< pgi <", pgi.upper_bound)

udcpdp = model.metabolites.get_by_id("udcpdp_c")
print( udcpdp.reactions )
print( udcpdp.formula )

Podemos verificar el balance de masa de una reacción:

In [ ]:
print(  "Balance de masa original" )
print(  pgi.reaction )
print(  pgi.check_mass_balance() )

print(  "Veamos que pasa con el balance de masa si ahora agregamos un metabolito a la reacción" )
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
print(  pgi.reaction )
print(  pgi.check_mass_balance() )

In [ ]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): 1,model.metabolites.get_by_id("atp_c"): 1})
print(  pgi.reaction )
print(  pgi.check_mass_balance() )

In [ ]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): 2})
print( pgi.reaction )
print( pgi.check_mass_balance() )

In [ ]:
print( pgi.check_mass_balance() )


Tambien podemos ver cuales son las reacciones en las que un determinado metabólito está involucrado:

In [ ]:
for rxn in model.metabolites.get_by_id("atp_c").reactions:
    if "BIOMASS" in rxn.id:
        print( rxn )

# Simulación de flujos metabólicos

Para simular los flujos metabólicos utilizando FBA (optimizamos la velocidad de producción de biomasa:

In [ ]:

model = cobra.test.create_test_model("ecoli")
solution=model.optimize()
print( model.summary() )
print( model.objective )

In [ ]:
print( model.objective )

In [ ]:
model.medium

In [ ]:
model.objective=model.reactions.get_by_id("EX_h_e")

In [ ]:
print( model.objective )
solution2 = model.optimize()
model.summary()

## Ejercicio

Simula y los flujos metabólicos de *E. coli* cuando:
    
1. El flujo de consumo de glucosa ("EX_glc__D_e") es igual a -2 [mmol/gDW/h]
2. El limite inferior para el flujo de consumo de oxigeno ("EX_o2_e") es igual a -10 [mmol/gDW/h]
